In [3]:
import os
import time
import multiprocessing as mp

file_name = "shared_log.txt"

def writer(tag):
    # Low-level append (process-safe open), but NOT record-safe
    fd = os.open(file_name, os.O_WRONLY | os.O_APPEND | os.O_CREAT)

    for i in range(30):
        os.write(fd, f"{tag} Record {i}: ".encode())
        time.sleep(0.005)
        os.write(fd, b"START | ")
        time.sleep(0.005)
        os.write(fd, b"END\n")
        time.sleep(0.001)

    os.close(fd)

if __name__ == "__main__":
    # reset file
    open(file_name, "w").close()

    p1 = mp.Process(target=writer, args=("A",))
    p2 = mp.Process(target=writer, args=("B",))

    p1.start()
    p2.start()
    p1.join()
    p2.join()

    print("Done. Open shared_log.txt and look for lines containing both A and B, or broken lines.")


Done. Open shared_log.txt and look for lines containing both A and B, or broken lines.


For Part 2, I simulated two different "users/programs" writing to the same log file at the same time. Each writer tries to save one record, but it writes the record in small pieces, with pauses in between. Because both writers are doing this at the same time, their pieces get mixed together in the file. That's why I see lines like A Record 0: B Record 0: START | START | END and also incomplete lines like A Record 1: END or a random START | END. As I increase the number of writes, the file gets more and more correupted because there are more chances for the writers to overlap. The file approach assumes records will stay together and only one writer will effectively write at the same time, but that assumption breaks once you have real concurrency.

**Key question answer:** This design assumes you won't have multiple writers hitting the file at the same time, which isn't true when usage grows.